# ch 15 RNN-coding
- http://pythonkim.tistory.com/57
- LSTM (Long Short Term Memory) 가장 대중적인 모델
- GRU (Gated recurrent unit) 우리나라 교수님의 모델

## 1. RNN in TensorFlow
- TensorFlow 0.9 이상에서는 코드가 아래와 같이 변경
- LSTM 공통 01 : Cell 생성
    1. rnn_cell = tf.nn.rnn_cell.<strong>BasicRNNCell</strong>(num_units = rnn_size, input_size = None, )
    1. rnn_cell = tf.nn.rnn_cell.<strong>BasicLSTMCell</strong>(num_units = rnn_size, input_size = None, )
    2. initial_state = rnn_cell.zero_state(batch_size, tf.float32)
    3. initial_state_1 = tf.zeros([batch_size, rnn_cell.state_size])
    4. x_split = tf.split(0, len(char_dic), x_data) 
- LSTM 공통 02 : Cell 구동
    5. outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)    

<img src="./img/RNNt-013.png" align="left">

In [1]:
#출력 사이즈를 정의
# hidden_size = 2     
# cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)

In [2]:
#입력 Dim을 정의
# x_data = np.array([[[1,0,0,0]]], dtype=np.float32)
# outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)

In [3]:
#출력모듈 실행
# sess.run(tf.global_variables_initializer())
# pp.print(outputs.eval())

## 2. Cell을 생성
- rnn_size : Output 의 크기값(4)을 입력 <p></p></br>
<img src="./img/RNNt-004.png" align="left" width=500>

- input : One-Hot Encoding ->  [[[1,0,0,0]]] shape=(1,1,4)
- Output : Hidden-size = 임의의 값 ->  [[[x,x]]] shape=(1,1,2)
- Input 데이터 형식과 , Hidden-size 만 '매개변수'로 입력하면 작동<p></p>
<img src ="./img/Lab12-RNN01.png" align="left" width=500>

<img src="./img/Lab12-RNNout.png" align="left" width = 700>

<img src="./img/Lab12-RNNout2.png" align="left" width = 700>

### 01. One cell RNN input_dim (4) -> output_dim (2)
- input = 5 ([1,0,0,0,0])  입력형태
- Sequence = 6 (연산모듈이 6개)
- hidden = 5 ([0,1,0,0,0]) 출력형태
- batch = 1 (현재는 1개로만 작동) <p></p></br>
<img src = "./img/lab12-Rnnpa.png" align="left" width = 700>

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [5]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [6]:
# One cell RNN input_dim (4) -> output_dim (2)
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2                                            # output_dim (2) 
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print('Output',cell.output_size, 'state',cell.state_size)  # 입력 데이터 형식을 출력
    x_data = np.array([[h]],  dtype=np.float32)                # x_data ( h = [[[1,0,0,0]]] )
    pp.pprint(x_data)                                          # 입력 데이터 출력
    outputs, _states = tf.nn.dynamic_rnn( cell, x_data, dtype=tf.float32 )
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())                                  # 최종 데이터 출력

Output 2 state 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[ 0.62225187, -0.40305367]]], dtype=float32)


In [7]:
# One cell RNN input_dim (4) -> output_dim (2). sequence: 5
with tf.variable_scope('two_sequances') as scope:
    hidden_size = 2                                            # output_dim (2)
    cell = tf.contrib.rnn.BasicRNNCell( num_units = hidden_size )
    x_data = np.array( [[h,e,l,l,o]], dtype=np.float32 )       # sequence: 5  cf) input_dim (4)
    print(x_data.shape)                                        # 입력 데이터 형식을 출력
    pp.pprint(x_data)                                          # 입력 데이터 출력
    outputs, states = tf.nn.dynamic_rnn( cell , x_data , dtype=tf.float32 )
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())                                  # 최종 데이터 출력

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[-0.32641998,  0.41991842],
        [-0.13929564, -0.48992422],
        [ 0.20226188, -0.24535619],
        [ 0.0399439 , -0.16477039],
        [-0.40884489,  0.5929662 ]]], dtype=float32)


In [8]:
# One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
with tf.variable_scope('3_batches') as scope:
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h,e,l,l,o], [e,o,l,l,l], [l,l,e,e,l]], dtype=np.float32)  
    pp.pprint(x_data)                                       # 입력 데이터 input_dim (4)
    hidden_size = 2                                         # 출력 데이터 output_dim (2)
    cell = rnn.BasicLSTMCell( num_units=hidden_size , state_is_tuple = True )
    outputs, _states = tf.nn.dynamic_rnn( cell , x_data , dtype=tf.float32 )
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())                               # 최종 데이터 출력

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.01180693,  0.07072996],
        [ 0.02780049, -0.01082219],
        [-0.06957481, -0.17919052],
        [-0.12637572, -0.25342694],
        [-0.00897235, -0.14294407]],

       [[ 0.0266323 , -0.06549848],
        [ 0.08703664, -0.00559421],
        [-0.03538973, -0.18263945],
        [-0.10124918, -0.25898033],
        [-0.13777752, -0.29646835]],

       [[-0.09290437, -0.16737527],
        [-0.14468664, -0.24425863],
        [-0.04393194, -0.18008718],
        [ 0.01278433, -0.19284461],
        [-0.06409218,

In [9]:
# One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
with tf.variable_scope('3_batches_dynamic_length') as scope:
    x_data = np.array([[h,e,l,l,o],[e,o,l,l,l],[l,l,e,e,l]], dtype=np.float32) # 3 batches 'hello', 'eolll', 'lleel'
    pp.pprint(x_data)    
    hidden_size = 2
    cell = rnn.BasicLSTMCell( num_units = hidden_size , state_is_tuple = True )
    # sequence_length=[5,3,4] : sequence: 5, batch 3, input_dim (4)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.10441136, -0.09090777],
        [ 0.07102746, -0.05561611],
        [-0.04449596, -0.18288326],
        [-0.13725717, -0.24952796],
        [-0.05420873, -0.13149051]],

       [[ 0.12674828, -0.0071988 ],
        [ 0.14043358,  0.03840204],
        [ 0.00520404, -0.11380652],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.10302107, -0.12369861],
        [-0.1797438 , -0.20011409],
        [ 0.00667668, -0.10969392],
        [ 0.13671795, -0.08314592],
        [ 0.        ,

In [10]:
# One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h,e,l,l,o], [e,o,l,l,l], [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(x_data)
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.07840708,  0.14941281],
        [-0.25659773,  0.09724767],
        [-0.25391352,  0.05390799],
        [-0.26038998,  0.05199654],
        [-0.14665331,  0.08220701]],

       [[-0.19065872, -0.00270398],
        [-0.07985987,  0.05199519],
        [-0.15294963,  0.0657903 ],
        [-0.19532792,  0.0701785 ],
        [-0.22303128,  0.07032412]],

       [[-0.08467411,  0.02964721],
        [-0.14667179,  0.04683939],
        [-0.28777272,  0.0343443 ],
        [-0.35641044, -0.00953914],
        [-0.30878997,

### 02. One cell RNN input_dim (3) -> output_dim (5)

In [11]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3
x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [12]:
# One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
with tf.variable_scope('generated_data') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[  1.48951430e-02,   1.36933044e-01,  -9.37520489e-02,
           1.44709498e-01,  -2.74909697e-02],
        [  1.23557728e-02,   2.21116468e-01,  -1.34138335e-02,
           5.41603625e-01,  -1.07498476e-02],
        [  6.71007764e-03,   1.97831646e-01,  -7.63019721e-04,
           7.18563378e-01,  -7.55928515e-04],
        [  3.40350647e-03,   1.25264287e-01,  -3.68362671e-05,
           7.66722083e-01,  -4.64893201e-05],
        [  1.64988311e-03,   6.80105537e-02,  -1.61077560e-06,
           7.73712277e-01,  -2.80107679e-06]],

       [[  2.59785721e-10,   1.48467673e-02,  -4.33666401e-08,
           7.49117076e-01,  -1.80648357e-07],
        [  1.30050137e-10,   1.42297512e-02,  -2.85983814e-09,
           7.67691970e-01,  -9.73030101e-09],
        [  6.04955738e-11,   7.86872301e-03,  -1.23809143e-10,
           7.65618324e-01,  -5.61959479e-10],
        [  2.80317471e-11,   3.90626397e-03,  -5.28892468e-12,
           7.63988674e-01,  -3.23283726e-11],
        [  1.2976

In [13]:
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers
    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[  3.85906035e-03,  -3.46304500e-03,   6.51773298e-04,
           3.47830844e-03,   3.02806986e-03],
        [  6.34685718e-03,  -9.81214643e-03,   1.92663574e-03,
           8.94505158e-03,   7.11261295e-03],
        [  4.56437608e-03,  -1.64472442e-02,   3.58432997e-03,
           1.46009829e-02,   1.04552554e-02],
        [ -9.86767700e-04,  -2.17805002e-02,   5.45538729e-03,
           1.94177721e-02,   1.27179381e-02],
        [ -8.88167135e-03,  -2.54344810e-02,   7.34295044e-03,
           2.30296962e-02,   1.40163219e-02]],

       [[ -1.22465356e-03,  -3.63395968e-03,  -2.73378973e-04,
           1.56903476e-03,   5.00238908e-04],
        [ -4.25911136e-03,  -8.40106979e-03,  -2.70955206e-04,
           3.64604429e-03,   1.32056267e-03],
        [ -8.89113080e-03,  -1.27781667e-02,   1.82134943e-04,
           5.33036236e-03,   2.27817846e-03],
        [ -1.44636370e-02,  -1.62092969e-

In [14]:
# lentgh 1 for batch 1, lentgh 2 for batch 2
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[  1.19335636e-01,   1.17564842e-01,  -5.99743389e-02,
           1.61135346e-01,  -1.34429395e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  3.36048943e-05,   2.53466794e-08,  -7.94968859e-04,
           1.59909114e-01,   4.94458480e-04],
        [  5.27783868e-06,   2.29055108e-09,  -2.49875098e-04,
           1.21128455e-01,   6.38624886e-04],
        [  7.92449896e-07,   1.72006756e-10,  -7.68404643e-05,
           8.67925659e-02,   6.87622756e-04],
        [  0.00000000e+00,   0.00000000e+0

In [15]:
# bi-directional rnn
with tf.variable_scope('bi-directional') as scope:
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ -1.30502075e-01,  -2.37174630e-02,  -9.64497998e-02,
           1.37160525e-01,  -1.54298857e-01],
        [ -3.71729225e-01,  -6.80332482e-02,  -1.48819208e-01,
           3.24346483e-01,  -3.21275324e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -5.32830894e-01,  -6.86394982e-04,  -7.80673327e-06,
           3.99099708e-01,  -3.77700984e-01],
        [ -8.35911155e-01,  -1.94387132e-04,  -8.87755778e-06,
           5.16412914e-01,  -5.56405425e-01],
        [ -9.43687737e-01,  -5.49392789e-05,  -9.01896692e-06,
           5.53423047e-01,  -6.12843633e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.

### 03. hidden_size=3 , sequence_length=5 , batch_size=3 , num_classes=5

In [16]:
# flattern based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

In [17]:
pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
       [ 42.,  43.,  44.]], dtype=float32)


In [18]:
softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[   25.,    28.,    31.,    34.,    37.],
        [   70.,    82.,    94.,   106.,   118.],
        [  115.,   136.,   157.,   178.,   199.],
        [  160.,   190.,   220.,   250.,   280.],
        [  205.,   244.,   283.,   322.,   361.]],

       [[  250.,   298.,   346.,   394.,   442.],
        [  295.,   352.,   409.,   466.,   523.],
        [  340.,   406.,   472.,   538.,   604.],
        [  385.,   460.,   535.,   610.,   685.],
        [  430.,   514.,   598.,   682.,   766.]],

       [[  475.,   568.,   661.,   754.,   847.],
        [  520.,   622.,   724.,   826.,   928.],
        [  565.,   676.,   787.,   898.,  1009.],
        [  610.,   730.,   850.,   970.,  1090.],
        [  655.,   784.,   913.,  1042.,  1171.]]], dtype=float32)


In [19]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])
# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)
# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

In [20]:
# sequence 데이터를 연산한다
# .sequence_loss( logits=예측값 , targets=목표값(seq) , weights= [,,,] 형태로 입력 )
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.596759


In [21]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)
prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

In [22]:
# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)
sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

In [23]:
sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval(),
      "Loss4: ", sequence_loss4.eval())

Loss1:  0.513015 Loss2:  0.371101 Loss3:  1.31326 Loss4:  1.31326


## 3. Cell을 구동
- rnn_size : Output 의 크기값(4)을 입력 <p></p></br>
<img src="./img/RNNt-004.png" align="left">

    outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)
- 세부 작동내용 확인 
- time Step size :  모델의 갯수 (여기선 rnn_size와 동일) </br></br><p>
<img src = "./img/RNNt-009.png" align="left">

    x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split

## 4. Cost Functions
### 01. Description

- logits, targets, weights 별로 사이즈를 맞춰야만 작동이 된다


    logits = tf.reshape(tf.concat(1, outputs), # shape = 1 x 16
                        [-1, rnn_size])        # shape = 4 x 4 (2D [batcha_size * num_decoder_symbols])
    targets = tf.reshape(ground_truth[1:], [-1]) # a shape of [-1] flattens into 1-D
    weights = tf.ones([len(char_dic) * batch_size])

- loss 함수 (개별적 모듈에 적용)
- cost 함수 (batch size 별로 적용)


    loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
    cost = tf.reduce_sum(loss) / batch_size
    train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

- Launch the graph in a session


    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(100):
            sess.run(train_op)
            result = sess.run(tf.argmax(logits, 1))
            print(result, [char_rdic[t] for t in result])    

### 02. Hi Hello Training

In [24]:
# Lab 12 RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [26]:
# 파라미터를 정의
num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

In [25]:
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
# x_data를 onehot으로 치환
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

In [27]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [28]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

In [29]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected( inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

In [30]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

In [31]:
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.65056 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
1 loss: 1.52608 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.45652 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.39797 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
4 loss: 1.31818 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
5 loss: 1.20625 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllll
6 loss: 1.08421 prediction:  [[1 0 0 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihhllo
7 loss: 0.973617 prediction:  [[1 0 0 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihhllo
8 loss: 0.872481 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihello
9 loss: 0.781866 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihel

In [2]:
import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[ 22.  28.]
 [ 49.  64.]]


In [4]:
# Creates a graph.
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [6]:
import torch
x = torch.cuda.FloatTensor(1)
# x.get_device() == 0
y = torch.FloatTensor(1).cuda()
# y.get_device() == 0

with torch.cuda.device(1):
    # allocates a tensor on GPU 1
    a = torch.cuda.FloatTensor(1)
    # transfers a tensor from CPU to GPU 1
    b = torch.FloatTensor(1).cuda()
    # a.get_device() == b.get_device() == 1
    c = a + b
    # c.get_device() == 1
    z = x + y
    # z.get_device() == 0
    # even within a context, you can give a GPU id to the .cuda call
    d = torch.randn(2).cuda(2)
    # d.get_device() == 2

RuntimeError: cuda runtime error (10) : invalid device ordinal at torch/csrc/cuda/Module.cpp:84

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from data_parallel import DataParallel

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=256, shuffle=True, num_workers=2, pin_memory=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

model = DataParallel(Net())
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.NLLLoss().cuda()

model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    input_var = Variable(data.cuda())
    target_var = Variable(target.cuda())

    print('Getting model output')
    output = model(input_var)
    print('Got model output')

    loss = criterion(output, target_var)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print('Finished')